<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/EMOS_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Εγκατάσταση απαιτούμενων βιβλιοθηκών
# pip install pandas pyarrow numpy pycountry

import pandas as pd
import numpy as np
#import pycountry

In [86]:
# Λήψη δεδομένων Eurostat (παράδειγμα dataset για ανεργία)
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/une_rt_m$defaultview/?format=TSV"

#Προσοχή στoν seperator(μπορεί να χρειάζονται περισσοτεροι από έναν)
#Στην περιπτωση μας /t , /
unemployment_data = pd.read_csv(url, sep=r'[,\t\\]')

# Προβολή δείγματος δεδομένων
print(unemployment_data.head(10))
unemployment_data.info()

<ipython-input-86-e8753735c16d>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  unemployment_data = pd.read_csv(url, sep=r'[,\t\\]')


  freq s_adj    age    unit sex   geo  TIME_PERIOD  2024-02   2024-03   \
0    M   NSA  TOTAL  PC_ACT   T    AT          5.5       5.4       4.9   
1    M   NSA  TOTAL  PC_ACT   T    BA         13.1      13.0      12.6   
2    M   NSA  TOTAL  PC_ACT   T    BE          5.7       5.6       5.4   
3    M   NSA  TOTAL  PC_ACT   T    BG          5.1       5.0       4.7   
4    M   NSA  TOTAL  PC_ACT   T    CH          4.3       4.2       4.0   
5    M   NSA  TOTAL  PC_ACT   T    CY          5.8       5.5       4.7   
6    M   NSA  TOTAL  PC_ACT   T    CZ          2.7       3.1       2.7   
7    M   NSA  TOTAL  PC_ACT   T    DE          3.6       3.5       3.2   
8    M   NSA  TOTAL  PC_ACT   T    DK          6.2       5.6       5.8   
9    M   NSA  TOTAL  PC_ACT   T  EA20          6.9       6.7       6.3   

   2024-04   2024-05   2024-06   2024-07   2024-08  2024-09  2024-10   2024-11  
0       5.1       4.8       4.7       5.3       5.4     5.1         :      NaN  
1      12.4      12.5  

In [87]:
# Επισκόπηση ελλείπουσων τιμών
missing_summary = unemployment_data.isnull().sum()
print(f"Missing Summary for rows:\n{missing_summary}")

# Διαγραφή γραμμών με πολλές ελλείπουσες τιμές
unemployment_data.dropna(how= "all",inplace=True)


# Συμπλήρωση ελλείπουσων τιμών με μέθοδο παρεμβολής
unemployment_data.interpolate(method='linear', inplace=True)

print( unemployment_data.head())

Missing Summary for rows:
freq             0
s_adj            0
age              0
unit             0
sex              0
geo              0
TIME_PERIOD      0
2024-02          0
2024-03          0
2024-04          0
2024-05          0
2024-06          0
2024-07          0
2024-08          0
2024-09          0
2024-10          0
2024-11        144
dtype: int64
  freq s_adj    age    unit sex geo  TIME_PERIOD  2024-02   2024-03   \
0    M   NSA  TOTAL  PC_ACT   T  AT          5.5       5.4       4.9   
1    M   NSA  TOTAL  PC_ACT   T  BA         13.1      13.0      12.6   
2    M   NSA  TOTAL  PC_ACT   T  BE          5.7       5.6       5.4   
3    M   NSA  TOTAL  PC_ACT   T  BG          5.1       5.0       4.7   
4    M   NSA  TOTAL  PC_ACT   T  CH          4.3       4.2       4.0   

   2024-04   2024-05   2024-06   2024-07   2024-08  2024-09  2024-10   2024-11  
0       5.1       4.8       4.7       5.3       5.4     5.1         :      NaN  
1      12.4      12.5      12.5      12.5  

<ipython-input-87-8cde12e148b1>:10: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  unemployment_data.interpolate(method='linear', inplace=True)


In [88]:
# Loop για όλες τις στηλες μετρώ τους χαρακτήρες ':' (Μετατρέπω ολες τις τιμες σε strings για τον ελεγχο)

for column in unemployment_data.columns:
    count_colon = unemployment_data[column].astype(str).str.contains(':', na=False).sum()
    print(f"Rows with ':' in '{column}': {count_colon}")


Rows with ':' in 'freq': 0
Rows with ':' in 's_adj': 0
Rows with ':' in 'age': 0
Rows with ':' in 'unit': 0
Rows with ':' in 'sex': 0
Rows with ':' in 'geo': 0
Rows with ':' in 'TIME_PERIOD': 0
Rows with ':' in '2024-02 ': 0
Rows with ':' in '2024-03 ': 0
Rows with ':' in '2024-04 ': 0
Rows with ':' in '2024-05 ': 0
Rows with ':' in '2024-06 ': 0
Rows with ':' in '2024-07 ': 0
Rows with ':' in '2024-08 ': 0
Rows with ':' in '2024-09 ': 16
Rows with ':' in '2024-10 ': 132
Rows with ':' in '2024-11': 0


In [89]:
#(διαγραφή στήλης 2024-11 και 2024-10 λογω ΝΑΝ και :)
unemployment_data.drop(columns=['2024-11','2024-10 '], inplace=True)


In [105]:
# Loop through each column
for column in unemployment_data.columns:
    # Αντικατάσταση':' με NaN
    unemployment_data[column] = unemployment_data[column].replace(':', float('nan'))

# Perform interpolation on the entire DataFrame
unemployment_data.interpolate(method='linear', inplace=True)

# ακομα εχω ':'. ########################
print(unemployment_data.head())


  freq s_adj    age    unit sex geo  TIME_PERIOD  2024-02   2024-03   \
0    M   NSA  TOTAL  PC_ACT   T  AT          5.5       5.4       4.9   
1    M   NSA  TOTAL  PC_ACT   T  BA         13.1      13.0      12.6   
2    M   NSA  TOTAL  PC_ACT   T  BE          5.7       5.6       5.4   
3    M   NSA  TOTAL  PC_ACT   T  BG          5.1       5.0       4.7   
4    M   NSA  TOTAL  PC_ACT   T  CH          4.3       4.2       4.0   

   2024-04   2024-05   2024-06   2024-07   2024-08  2024-09   
0       5.1       4.8       4.7       5.3       5.4     5.1   
1      12.4      12.5      12.5      12.5      12.3       :   
2       5.3       5.3       5.8       6.0       6.0     5.9   
3       4.2       3.9       3.7       3.6       3.5     3.6   
4       3.9       4.0       4.5       4.8       4.9       :   


<ipython-input-105-174e0dbfb23d>:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  unemployment_data.interpolate(method='linear', inplace=True)


In [92]:
# Check the data types of the columns
print(unemployment_data.dtypes)


freq            object
s_adj           object
age             object
unit            object
sex             object
geo             object
TIME_PERIOD    float64
2024-02        float64
2024-03        float64
2024-04        float64
2024-05        float64
2024-06        float64
2024-07        float64
2024-08        float64
2024-09         object
dtype: object


In [127]:
#Checking again the columns only
print(unemployment_data.columns)

#Παρατηρώ πως ο καθορισμός χρειάζεται να γίνει σε στήλες που υπάρχουν και
# να είναι αριθμητικές
#Μαζέυω όλες τις στήλες που περιέχουν αριθμό(δηλαδή τους μήνες)
numeric_columns = unemployment_data.select_dtypes(include='number').drop(columns=['TIME_PERIOD'], errors='ignore')

# Inspect the resulting DataFrame
print(numeric_columns.head())
print( unemployment_data.head())

Index(['freq', 's_adj', 'age', 'unit', 'sex', 'geo', 'TIME_PERIOD', '2024-02 ',
       '2024-03 ', '2024-04 ', '2024-05 ', '2024-06 ', '2024-07 ', '2024-08 ',
       '2024-09 '],
      dtype='object')
   2024-02   2024-03   2024-04   2024-05   2024-06   2024-07   2024-08 
0       5.4       4.9       5.1       4.8       4.7       5.3       5.4
1      13.0      12.6      12.4      12.5      12.5      12.5      12.3
2       5.6       5.4       5.3       5.3       5.8       6.0       6.0
3       5.0       4.7       4.2       3.9       3.7       3.6       3.5
4       4.2       4.0       3.9       4.0       4.5       4.8       4.9
  freq s_adj  age unit  sex  geo  TIME_PERIOD  2024-02   2024-03   2024-04   \
0  NaN   NaN  NaN  NaN  NaN  NaN          5.5       5.4       4.9       5.1   
1  NaN   NaN  NaN  NaN  NaN  NaN         13.1      13.0      12.6      12.4   
2  NaN   NaN  NaN  NaN  NaN  NaN          5.7       5.6       5.4       5.3   
3  NaN   NaN  NaN  NaN  NaN  NaN          5.1      

In [110]:
# Καθορισμός ορίων για ακραίες τιμές (π.χ., 1.5 IQR)
numeric_columns = unemployment_data.select_dtypes(include='number').columns
q1 = unemployment_data[numeric_columns].quantile(0.25)
q3 = unemployment_data[numeric_columns].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Αντικατάσταση ακραίων τιμών με NaN
for col in numeric_columns:
    unemployment_data[col] = unemployment_data[col].apply(
        lambda x: np.nan if x < lower_bound[col] or x > upper_bound[col] else x
    )


In [111]:
print(unemployment_data.head())
# Αντικατάσταση ακραίων τιμών με NaN
#unemployment_data[unemployment_data.columns] = unemployment_data[unemployment_data.columns].apply(
#    lambda x: np.nan if x < lower_bound or x > upper_bound else x
#)
unemployment_data.columns

  freq s_adj    age    unit sex geo  TIME_PERIOD  2024-02   2024-03   \
0    M   NSA  TOTAL  PC_ACT   T  AT          5.5       5.4       4.9   
1    M   NSA  TOTAL  PC_ACT   T  BA         13.1      13.0      12.6   
2    M   NSA  TOTAL  PC_ACT   T  BE          5.7       5.6       5.4   
3    M   NSA  TOTAL  PC_ACT   T  BG          5.1       5.0       4.7   
4    M   NSA  TOTAL  PC_ACT   T  CH          4.3       4.2       4.0   

   2024-04   2024-05   2024-06   2024-07   2024-08  2024-09   
0       5.1       4.8       4.7       5.3       5.4     5.1   
1      12.4      12.5      12.5      12.5      12.3       :   
2       5.3       5.3       5.8       6.0       6.0     5.9   
3       4.2       3.9       3.7       3.6       3.5     3.6   
4       3.9       4.0       4.5       4.8       4.9       :   


Index(['freq', 's_adj', 'age', 'unit', 'sex', 'geo', 'TIME_PERIOD', '2024-02 ',
       '2024-03 ', '2024-04 ', '2024-05 ', '2024-06 ', '2024-07 ', '2024-08 ',
       '2024-09 '],
      dtype='object')

In [112]:
# Τυποποίηση κατηγορικών τιμών (π.χ., geo, sex, age)
unemployment_data['geo'] = unemployment_data['geo'].str.upper().str.strip()
unemployment_data['sex'] = unemployment_data['sex'].str.capitalize().str.strip()
unemployment_data['age'] = unemployment_data['age'].str.replace(" ", "_")

In [113]:
# Εντοπισμός και αφαίρεση διπλών εγγραφών
unemployment_data.drop_duplicates(inplace=True)

In [117]:
# Συμπλήρωση ελλειπουσών τιμών με μέσο όρο ανά κατηγορία
unemployment_data[numeric_columns] = unemployment_data.groupby(['geo', 'sex', 'age'])[numeric_columns].transform(
    lambda x: x.fillna(x.mean())
)

In [115]:
# Καθαρισμός ανεπιθύμητων χαρακτήρων στις στήλες
unemployment_data['geo'] = unemployment_data['geo'].str.strip()
unemployment_data['sex'] = unemployment_data['sex'].str.strip()
unemployment_data['age'] = unemployment_data['age'].str.strip()

In [118]:
# Αφαίρεση αρνητικών και μη λογικών τιμών
unemployment_data = unemployment_data[unemployment_data[numeric_columns] >= 0]

In [121]:
# Μετατροπή τιμών σε σωστές μορφές
#unemployment_data[numeric_columns] = pd.to_numeric(unemployment_data[numeric_columns], errors='coerce')
#unemployment_data['TIME_PERIOD'] = pd.to_datetime(unemployment_data['TIME_PERIOD'])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [123]:
# Φιλτράρισμα δεδομένων με βάση λογικά όρια (0-100 για ποσοστά)
unemployment_data = unemployment_data[(unemployment_data[numeric_columns] >= 0) & (unemployment_data[numeric_columns] <= 100)]

In [125]:
# Αποθήκευση των καθαρισμένων δεδομένων σε αρχείο CSV
unemployment_data.to_csv("cleaned_unemployment_data.csv", index=False)

# Έλεγχος αποθήκευσης
loaded_csv_data = pd.read_csv("cleaned_unemployment_data.csv")
print(loaded_csv_data.head())

# Αποθήκευση σε αρχείο Parquet
unemployment_data.to_parquet("cleaned_unemployment_data.parquet", index=False)

# Έλεγχος αποθήκευσης
loaded_data = pd.read_parquet("cleaned_unemployment_data.parquet")
print(loaded_data.head())


   freq  s_adj  age  unit  sex  geo  TIME_PERIOD  2024-02   2024-03   \
0   NaN    NaN  NaN   NaN  NaN  NaN          5.5       5.4       4.9   
1   NaN    NaN  NaN   NaN  NaN  NaN         13.1      13.0      12.6   
2   NaN    NaN  NaN   NaN  NaN  NaN          5.7       5.6       5.4   
3   NaN    NaN  NaN   NaN  NaN  NaN          5.1       5.0       4.7   
4   NaN    NaN  NaN   NaN  NaN  NaN          4.3       4.2       4.0   

   2024-04   2024-05   2024-06   2024-07   2024-08   2024-09   
0       5.1       4.8       4.7       5.3       5.4       NaN  
1      12.4      12.5      12.5      12.5      12.3       NaN  
2       5.3       5.3       5.8       6.0       6.0       NaN  
3       4.2       3.9       3.7       3.6       3.5       NaN  
4       3.9       4.0       4.5       4.8       4.9       NaN  
   freq s_adj   age  unit   sex   geo  TIME_PERIOD  2024-02   2024-03   \
0  None  None  None  None  None  None          5.5       5.4       4.9   
1  None  None  None  None  None  No